In [1]:
# Now we'll do the same task as before but now we're gonna use a pre trained model called VGG16 that was trained on the 
# dataset ImageNet 
# Basically we're gonna remove the top model(fully connected layer)-just use the convulotion layers -and retrain the model just on our data one time 
# then we're gonna save those learned features (called bottleneck features) 
# then we're gonna train custome fully connected layers on top of our previously stored features !! 

## steps !! 
# 1. of course we first import our libraries and specify our constants !! 

# 2. then we create our image generator to just rescale/ normalize the data 

# 3. then we import the pretrained model 

# 4. And now we get the data ready for the model but just the data not the labels because we're gonna disable 
# the shuffle option and now we'd have 1000 cat then 1000 dog

# 5. now we prepare the train data and labels and val data and labels then we construct our model architecture 


In [2]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation
from keras import applications
import os 

Using TensorFlow backend.


In [3]:
cwd = os.getcwd()
Height, Width = 150, 150 
epochs = 50 
batch_size = 16 
m1 = 2000
m2 = 800 
train_dir = os.path.join(cwd, 'kerasClassifier/train')
val_dir = os.path.join(cwd, 'kerasClassifier/validation')
pretrained_weights_path = os.path.join(cwd, 'vgg16_weights.h5') 
top_model_weights_path = 'bottleneck_fc_model.h5'

In [4]:
def save_bottelneck_features():
    
    datagen = ImageDataGenerator(rescale = 1. / 255)
    
    model = applications.VGG16(include_top=False, weights='imagenet')
    
    generator = datagen.flow_from_directory(train_dir, 
                                                  target_size = (Height, Width),
                                                  batch_size = batch_size, 
                                                  class_mode = None,
                                                  shuffle = False)
    bottleneck_features_train = model.predict_generator(generator, m1 // batch_size)
    np.save(open('bottleneck_features_train.npy', 'wb'), bottleneck_features_train)
    
    
    generator = datagen.flow_from_directory(val_dir,
                                                target_size = (Height, Width),
                                                batch_size = batch_size, 
                                                class_mode = None,
                                                shuffle = False)
    bottleneck_features_val = model.predict_generator(generator, m2 // batch_size)
    np.save(open('bottleneck_features_val.npy', 'wb'), bottleneck_features_val)
    
    

In [5]:
def train_top_model():
    
    train_data = np.load(open('bottleneck_features_train.npy', 'rb'))
    train_labels = np.array( [0]*(m1//2) + [1]*(m1//2) ) 
    
    val_data = np.load(open('bottleneck_features_val.npy', 'rb'))
    val_labels = np.array( [0]*(m2//2) + [1]*(m2//2) ) 
    
    model = Sequential()
    model.add(Flatten(input_shape = train_data.shape[1:])) #train_data[1:] to omit the number of examples fromt the shape 
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5)) 
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    model.compile(optimizer = 'rmsprop',
                  loss = 'binary_crossentropy',
                  metrics = ['accuracy'])
    
    model.fit(train_data,train_labels, 
              batch_size = batch_size,
              epochs = epochs,
              validation_data = (val_data, val_labels))
    
    model.save_weights(top_model_weights_path)


    
    
    

In [6]:
save_bottelneck_features()



Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [ ]:
train_top_model()

Train on 2000 samples, validate on 800 samples
Epoch 1/50
2000/2000 [==============================] - 9s 4ms/step - loss: 0.8117 - acc: 0.7520 - val_loss: 0.2924 - val_acc: 0.8762
Epoch 2/50
2000/2000 [==============================] - 7s 3ms/step - loss: 0.3431 - acc: 0.8535 - val_loss: 0.2892 - val_acc: 0.8788
Epoch 3/50
2000/2000 [==============================] - 7s 4ms/step - loss: 0.2856 - acc: 0.8905 - val_loss: 0.2473 - val_acc: 0.9075
Epoch 4/50
2000/2000 [==============================] - 7s 4ms/step - loss: 0.2472 - acc: 0.9065 - val_loss: 0.2874 - val_acc: 0.9025
Epoch 5/50
2000/2000 [==============================] - 7s 3ms/step - loss: 0.2399 - acc: 0.9130 - val_loss: 0.2868 - val_acc: 0.8825
Epoch 6/50
2000/2000 [==============================] - 6s 3ms/step - loss: 0.1959 - acc: 0.9260 - val_loss: 0.4226 - val_acc: 0.8562
Epoch 7/50
2000/2000 [==============================] - 7s 3ms/step - loss: 0.1742 - acc: 0.9355 - val_loss: 0.3948 - val_acc: 0.8725
Epoch 8/50
1504